<a href="https://colab.research.google.com/github/SophiaU/Git_DoD_Remittance/blob/main/DoD_Reconciliation_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install and import pygsheets
!pip install pygsheets
import pygsheets

#  Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# connect google sheets
import gspread
# import pygsheets
gc = gspread.authorize(GoogleCredentials.get_application_default())


# import json
# from google.oauth2 import service_account

# with open('service_account.json') as source:
#    info = json.load(source)
# credentials = service_account.Credentials.from_service_account_info(info)

# client = pygsheets.authorize(service_account_file='service_account.json')


# import pandas
import pandas as pd
from functools import reduce


     |████████████████████████████████| 153kB 5.3MB/s 


In [ ]:
weekFolderID = '1NS4RyHkmE6LHxoar3fS52Yxyp81C_BIK'

Combine Performance File

In [ ]:
# Append all files in folder into a listframe

listframe = []

weekFolder = drive.ListFile({'q': "'"+weekFolderID+"' in parents and  mimeType = 'application/vnd.google-apps.folder'"}).GetList()


for folder in weekFolder:
  if folder['title'] == 'Performance':
    files = drive.ListFile({'q': "not title contains '(7)' and not title contains '(6)' and '"+folder['id']+"' in parents and mimeType = 'text/csv' and trashed = false "}).GetList()
    for file in files:
      print(file['title'])
      # download
      fileDownloaded = drive.CreateFile({'id':file['id']})

      # load
      fileDownloaded.GetContentFile('performance.csv')

      # read
      df = pd.read_csv('performance.csv', delimiter=',' )

      # append
      listframe.append(df)

# combine all files in the list into a dataframe
wkperffile = pd.concat(listframe)

# pick only name, trips and supply hours
performancefile = wkperffile[['Name','Trips','Hours online']]
performancefile['Name'] = pd.Series(performancefile['Name']).str.title()

# convert hour online to just hours
performancefile['Hours online'] = pd.to_timedelta(performancefile['Hours online']).dt.total_seconds()/3600

# sum trips and hours by names
performancefile = performancefile.groupby(['Name']).sum().reset_index()
performancefile

driver_performance Jul 15(5).csv
driver_performance Jul 15(4).csv
driver_performance Jul 15(3).csv
driver_performance Jul 15(2).csv
driver_performance Jul 15(1).csv
driver_performance Jul 14(5).csv
driver_performance Jul 14(4).csv
driver_performance Jul 14(2).csv
driver_performance Jul 14(3).csv
driver_performance Jul 14(1).csv
driver_performance Jul 12(1).csv
driver_performance Jul 12(2).csv
driver_performance Jul 12(3).csv
driver_performance Jul 12(4).csv
driver_performance Jul 12(5).csv
driver_performance Jul 13(5).csv
driver_performance Jul 13(1).csv
driver_performance Jul 13(4).csv
driver_performance Jul 13(3).csv
driver_performance Jul 13(2).csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Name,Trips,Hours online
0,Abimbola Omojuwa,33,36.9
1,Abiodun Adedeji,34,48.7
2,Abiodun Onamosu,48,46.3
3,Abiodun Sunday Adisa,31,32.9
4,Adebayo Muyideen Alimi,36,35.5
...,...,...,...
340,Williams Aikhuemero,32,42.2
341,Williams Nwadinobi Anunukem,32,40.4
342,Wisdom Akpan,39,49.4
343,Yetunde Samiat Shobowale,13,19.5


In [ ]:
#  Append all files in folder into a listframe
paymentslistframe = []

for folder in weekFolder:
  if folder['title'] == 'Payments':
    files = drive.ListFile({'q': "not title contains '(7)' and not title contains '(6)' and '"+folder['id']+"' in parents and mimeType = 'text/csv' and trashed = false "}).GetList()
    for file in files:
      # print(file['title'])

      # download
      fileDownloaded = drive.CreateFile({'id':file['id']})

      # load
      fileDownloaded.GetContentFile('payments.csv')

      # read
      df = pd.read_csv('payments.csv', delimiter=',' )

      # append
      paymentslistframe.append(df)

# combine all files in the list into a dataframe
combinedPayments = pd.concat(paymentslistframe)

# concatenate the names
combinedPayments['Name'] = combinedPayments['firstName'] + " " + combinedPayments['lastName']
combinedPayments['Name'] = pd.Series(combinedPayments['Name']).str.title()
combinedPayments = combinedPayments[['Name','driverUUID', 'tripUUID', 'firstName', 'lastName', 'amount','timestamp', 'itemType', 'description', 'disclaimer']]

# pick only fullname, amount and itemtype
paymentsfile = combinedPayments[['Name','driverUUID','amount','itemType']]

# get net earnings
# drop cash collected itemtype
netEarnings = paymentsfile.loc[paymentsfile['itemType'] != 'cash_collected']

# drop payouts itemtype
netEarnings = netEarnings.loc[netEarnings['itemType'] != 'payouts']

# sum amounts by names to get the net earnings per driver
netEarnings = netEarnings.groupby(['driverUUID']).sum().reset_index()
netEarnings.columns = ['driverUUID','Net Earnings']
# netEarnings['Name'] = pd.Series(netEarnings['Name']).str.title()
netEarnings.head(400)

# get total cash collected
# get cash collected itemtype
cashCollected = paymentsfile.loc[paymentsfile['itemType'] == 'cash_collected']

# sum amounts by names to get the cash collected per driver
cashCollected = cashCollected.groupby(['driverUUID']).sum().reset_index()
cashCollected.columns = ['driverUUID', 'Cash Collected']
# cashCollected['Name'] = pd.Series(cashCollected['Name']).str.title()
cashCollected.head(400)

# combinedPayments.head(400)



,driverUUID,Cash Collected
0,001319fb-c6f6-4849-a062-515e59dc0689,-63300.0
1,0073465c-8143-49c6-a37a-4f0b29738c05,-27900.0
2,0078ddf7-9f70-4fc1-bf93-80a5114dfbfc,-8400.0
3,011bfaae-ef9c-40c4-b49a-5acfc3f5e56c,-43200.0
4,01ef8975-0e8c-41a1-982e-f3ded75449a5,-32400.0
...,...,...
339,fb2060b2-b6a4-4e59-b411-a6fa2c5623a0,-44100.0
340,fb49c327-2bac-4479-8866-fad76b89e207,-50300.0
341,fbbb9e0a-dd81-4778-866d-fc162755f26f,-17400.0
342,fce722ca-e6db-456e-842d-ea22857d4cb1,-11200.0


In [ ]:
#Add Driver Uber ID to Performance file

uniqueID = combinedPayments.groupby(['Name','driverUUID']).sum().reset_index()

# uniqueID = combinedPayments.drop_duplicates(
#   subset = ['Name', 'driverUUID'],
#   keep = 'last').reset_index(drop = True)

performancefile1 = pd.merge(performancefile, uniqueID[['Name','driverUUID']], on='Name', how='left')
performancefile1 = performancefile1[['Name','Trips','Hours online','driverUUID']]
performancefile1 = performancefile1.groupby(['driverUUID']).sum().reset_index()
performancefile1


# NameCheck = performancefile1.query('driverUUID == "de90e988-5c1d-4ee7-ae7d-0a1418cdd111"')
# NameCheck = performancefile1.query('Name == "Onyeka Aigbaze"')     # Isiaq Jamiu Akanbi
# NameCheck


,driverUUID,Trips,Hours online
0,001319fb-c6f6-4849-a062-515e59dc0689,40,51.1
1,0073465c-8143-49c6-a37a-4f0b29738c05,22,31.2
2,0078ddf7-9f70-4fc1-bf93-80a5114dfbfc,4,5.3
3,011bfaae-ef9c-40c4-b49a-5acfc3f5e56c,40,54.2
4,01ef8975-0e8c-41a1-982e-f3ded75449a5,48,35.1
...,...,...,...
339,fb2060b2-b6a4-4e59-b411-a6fa2c5623a0,30,47.9
340,fb49c327-2bac-4479-8866-fad76b89e207,49,36.0
341,fbbb9e0a-dd81-4778-866d-fc162755f26f,28,28.0
342,fce722ca-e6db-456e-842d-ea22857d4cb1,8,9.2


In [ ]:
performancefile1.head(400)

,driverUUID,Trips,Hours online
0,001319fb-c6f6-4849-a062-515e59dc0689,40,51.1
1,0073465c-8143-49c6-a37a-4f0b29738c05,22,31.2
2,0078ddf7-9f70-4fc1-bf93-80a5114dfbfc,4,5.3
3,011bfaae-ef9c-40c4-b49a-5acfc3f5e56c,40,54.2
4,01ef8975-0e8c-41a1-982e-f3ded75449a5,48,35.1
...,...,...,...
339,fb2060b2-b6a4-4e59-b411-a6fa2c5623a0,30,47.9
340,fb49c327-2bac-4479-8866-fad76b89e207,49,36.0
341,fbbb9e0a-dd81-4778-866d-fc162755f26f,28,28.0
342,fce722ca-e6db-456e-842d-ea22857d4cb1,8,9.2


Get Moove Details

In [ ]:
# get moove details
mooveDetailList =[]
worksheet = gc.open_by_key('1Q3BJcXwRncZiiB3GAYw1CNN-RWOWjt4AyUHxjs_Kztg').sheet1
for i in range(2,6):
  columns = worksheet.col_values(i)
  mooveDetailList.append(columns[1:])

# print(mooveDetailList)

# convert list to data frame
mooveDetails = pd.DataFrame(mooveDetailList)
mooveDetails = mooveDetails.T
mooveDetails.columns = ['Name','Product','DRN','driverUUID']
mooveDetails['Name'] = pd.Series(mooveDetails['Name']).str.title()
#print(mooveDetails)
mooveDetails

,Name,Product,DRN,driverUUID
0,Inwang Ubong,income product,DRN000022,4ed6faed-e165-4a8b-8e47-9a24433ddb91
1,Adeola Okusaga,income product,DRN000220,b2e4d5a7-fbae-49a2-be3a-4ba1b9f179f5
2,Temitope Tayo,income product,DRN000229,
3,Chiegeokwu Amaju,flexi rental,DRN000230,0f23e379-8b69-4233-9497-0ad579a2f60a
4,Adanna Cythia Nwanna,flexi rental,DRN000233,0af65713-855a-4a7f-8d15-5fca9dbfecbb
...,...,...,...,...
930,Paul Akere Omokaro,DTO-II 48,DRN002663,
931,Kelechi Oluwatoba Okorie,DTO-II 48,DRN002662,ba59a846-4044-4fae-84c6-81a9628fd6b7
932,Omololu Kehinde Sotonwa,DTO-II 48,DRN002633,a1660b79-4752-4dfd-a79b-3bc14db3163d
933,Aimuanmwosa Amos Asemota,DTO-II 48,DRN002654,


Virtual Accounts Data

In [ ]:
fileid = '1DRk8HYz_HPu_K8jzGEQB_N2hu5StgaWo'

folder = drive.ListFile({'q': "'"+fileid+"' in parents and mimeType = 'text/csv' and trashed = false "}).GetList()
for file in folder:
  # print(file['title'])

  # download
  fileDownloaded = drive.CreateFile({'id':file['id']})

  # load
  fileDownloaded.GetContentFile('VirtualAccount.csv')

  # read
  VirtualAccountDF = pd.read_csv('VirtualAccount.csv', delimiter=',' )

# select only drn and amount
VirtualAcctDF = VirtualAccountDF[['drn','amount']]  
VirtualAcctDF.rename(columns={'drn':'DRN'}, inplace=True)

# Sum Amount
sumVA = VirtualAcctDF.groupby(['DRN']).sum().reset_index()
sumVA.rename(columns={'amount':'VA Collections'}, inplace=True)

# count amount
countVA = VirtualAcctDF.groupby(['DRN']).count().reset_index()
countVA.rename(columns={'amount':'VA Transactions'}, inplace=True)

print(sumVA)
print(countVA)

          DRN  VA Collections
0   DRN000466           14300
1   DRN000554           15800
2   DRN000574           16760
3   DRN000603            1700
4   DRN000665           24300
5   DRN000762           10800
6   DRN000924             700
7   DRN001060            6868
8   DRN001086            1600
9   DRN001093            2100
10  DRN001112           18700
11  DRN001186           18900
12  DRN001274            1700
13  DRN001312            1300
14  DRN001482            1400
15  DRN001511           14900
16  DRN001534           17900
17  DRN001623            2000
18  DRN001649            7700
19  DRN001751            4400
20  DRN001778            1400
21  DRN001875           12000
22  DRN001957            2300
23  DRN002005           12050
24  DRN002089           18800
25  DRN002172           10300
26  DRN002283            1200
          DRN  VA Transactions
0   DRN000466                6
1   DRN000554                7
2   DRN000574               23
3   DRN000603                1
4   D

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Days with Car Data

In [ ]:
# get moove details
daysWithCarlList =[]
carsWorksheet = gc.open_by_key('1_hFrUFt5qoXbah9tLyQLnbmTZFPzeySYGYBnPjE3eZQ').sheet1
for i in range(10,12):
  columns = carsWorksheet.col_values(i)
  daysWithCarlList.append(columns[1:])

# print(mooveDetailList)

# convert list to data frame
daysWithCar = pd.DataFrame(daysWithCarlList)
daysWithCar = daysWithCar.T
daysWithCar.columns = ['DRN','Effective Days']
daysWithCar['DRN'] = pd.Series(daysWithCar['DRN']).str.strip()

# Borrowed from weekly script
# daysWithCar = pd.merge(daysWithCar,mooveDetails,how='left', on='DRN')
daysWithCar
#print(daysWithCar)

,DRN,Effective Days
0,DRN001222,1
1,DRN000385,2
2,DRN001758,1
3,DRN001693,3
4,DRN000504,2
...,...,...
360,DRN002503,-1
361,DRN002652,-1
362,DRN002296,-1
363,DRN002658,-1


Combine Performance, Earnings and Moove Details

In [ ]:
# join elements of the final payments file
dfs = [performancefile1,netEarnings, cashCollected, mooveDetails]

# join on Name
reconfile = reduce(lambda left,right: pd.merge(left,right,how='left', on='driverUUID'), dfs)

# join on DRN
dfs = [reconfile,sumVA, countVA,daysWithCar]
reconfile = reduce(lambda left,right: pd.merge(left,right,how='left', on='DRN'), dfs)

reconfile['Cash Collected'] = -reconfile['Cash Collected']
reconfile['Uber Balance'] = reconfile['Net Earnings'] - reconfile['Cash Collected']
reconfile['VA Transactions'] = reconfile['VA Transactions'].fillna(0).astype(int)
reconfile['VA Collections'] = reconfile['VA Collections'].fillna(0)
reconfile['VA Incentive'] = pd.DataFrame(map(lambda x: 7000 if x >= 20 else x * 100, reconfile['VA Transactions']))

reconfile = reconfile[['DRN', 'Name', 'Product', 'Uber Balance', 'Cash Collected', 'Net Earnings', 
        'Trips', 'Hours online','Effective Days','driverUUID']]
# reconfile['Name'] = pd.Series(reconfile['Name']).str.title()
reconfile.head(400)



,DRN,Name,Product,Uber Balance,Cash Collected,Net Earnings,Trips,Hours online,Effective Days,driverUUID
0,DRN001694,Ncheta Ewuzie,DTO 48,7082.25,63300.0,70382.25,40,51.1,3,001319fb-c6f6-4849-a062-515e59dc0689
1,DRN000423,Opeyemi Raheem,DTO 30,7510.50,27900.0,35410.50,22,31.2,3,0073465c-8143-49c6-a37a-4f0b29738c05
2,DRN002312,Oluwatobi Adeyanju,DTO 30,340.50,8400.0,8740.50,4,5.3,3,0078ddf7-9f70-4fc1-bf93-80a5114dfbfc
3,DRN000594,Cute Sagay,income product,14992.25,43200.0,58192.25,40,54.2,3,011bfaae-ef9c-40c4-b49a-5acfc3f5e56c
4,DRN000667,Promise Akpukpu,DTO 48,12105.75,32400.0,44505.75,48,35.1,3,01ef8975-0e8c-41a1-982e-f3ded75449a5
...,...,...,...,...,...,...,...,...,...,...
339,DRN001535,Paul Osebitu Mosusu,income product,8870.25,44100.0,52970.25,30,47.9,3,fb2060b2-b6a4-4e59-b411-a6fa2c5623a0
340,DRN000558,Taiwo Bamisaiye,income product,20108.50,50300.0,70408.50,49,36.0,3,fb49c327-2bac-4479-8866-fad76b89e207
341,DRN000797,Olalekan Lawal Bello,DTO 48,4381.50,17400.0,21781.50,28,28.0,3,fbbb9e0a-dd81-4778-866d-fc162755f26f
342,DRN001957,Olivia Aniedozie,flexi rental,1586.75,11200.0,12786.75,8,9.2,3,fce722ca-e6db-456e-842d-ea22857d4cb1


Upload File to Google Drive

In [ ]:
# # export to csv
reconfile_csv = reconfile.to_csv( "reconfile_15thJul.csv", index=False, encoding='utf-8-sig')

file2 = drive.CreateFile({'parents': [{'id': '15w_MeKLpCE2hs8-Vvs4IUBH_sZ8e3qbP'}]})
file2.SetContentFile("reconfile_15thJul.csv")
file2.Upload()

In [ ]:
# #  export to csv
paymentsCombined_csv = combinedPayments.to_csv( "paymentsfile_15thJul.csv", index=False, encoding='utf-8-sig')

file2 = drive.CreateFile({'parents': [{'id': '15w_MeKLpCE2hs8-Vvs4IUBH_sZ8e3qbP'}]})
file2.SetContentFile("paymentsfile_15thJul.csv")
file2.Upload()

Write reconciled files into google sheets

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving serviceaccount.json to serviceaccount.json


In [ ]:
import json
from google.oauth2 import service_account

with open('serviceaccount.json') as source:
    info = json.load(source)
credentials = service_account.Credentials.from_service_account_info(info)

client = pygsheets.authorize(service_account_file='serviceaccount.json')

In [ ]:
reconSheet = client.open_by_key('1r6_Kn4mFKhfSrFKShJN7ouBSNuCQ8cfhVZTlKsU_wo8')
sheet1 = reconSheet[0]

sheet1.set_dataframe(reconfile,(1,1))

In [ ]:
# create new sheet for the week

# weekname = "May 11"
# sh = gc.open_by_key('1Q3BJcXwRncZiiB3GAYw1CNN-RWOWjt4AyUHxjs_Kztg')
# weeksheet = sh.add_worksheet(title=weekname, rows="100", cols="10")

In [ ]:
# for item in dir(wks):
#   if not item.startswith("_"):
#     print(item)

In [ ]:
# weekperformance = pd.merge(wkperffile,mooveDetails, how='left', on='Name')
# weekperformance = weekperformance[['DRN', 'Name',  'Product', 'Net Fares', 'Per trip', 'Per hour online', 'Per km on trip',
#        'Trips', 'Hours online', 'Trips per hour', 'Distance per trip',
#        'Ratings', 'Lifetime rating', 'Acceptance rate',
#        'Driver cancellation rate' ]]
# # worksheetrange = weeksheet.range('B1:P1')
# weeksheet.set_dataframe(weekperformance, start=(1,1))

In [ ]:
# combinedPayments = combinedPayments[['Name','driverUUID', 'tripUUID', 'firstName', 'lastName', 'amount',
#        'timestamp', 'itemType', 'description', 'disclaimer']]
# dtoearnings = client.open_by_key('1ijlQsyalGJdHesighUmYFgzQYEELLQS6ZxixT9cLszo')
# wks = dtoearnings[1]

# wks.set_dataframe(combinedPayments,(1,2))

In [ ]:
wks.get_col(2,include_tailing_empty=False).count?

Object `count` not found.
